<a href="https://colab.research.google.com/github/Cathy-Guang/TB-drug-resistance/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import csv

In [86]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'TB-drug-resistance/' #change this to whatever folder your code is in
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

GROUND_TRUTH_PATH = os.path.join(GOOGLE_DRIVE_PATH, "test_data/ground_truth")
RESULTS_PATH = os.path.join(GOOGLE_DRIVE_PATH, "results/")


Mounted at /content/drive


In [88]:
models = ["SLE_results","SVM_results","XGBoost_results","logistic_regression_results"]
data_process = ["PCA","SMOTE","column_modified"]

drugs = ['AMK','CAP','EMB','INH','KAN','MOXI','OFLX','PZA','RIF','STR']

field = ["Model","Drug","Accuracy","Precision","Recall", "rocauc"]
evaluation_list = []
evaluation_list.append(field)

for drug in drugs:
  for model in models:
    for method in data_process:

      truth_file = pd.read_csv(os.path.join(GROUND_TRUTH_PATH, 'Y_testData_1_results_'+drug+'.csv'))
      truth = truth_file[drug].to_numpy()

      result_file = pd.read_csv(os.path.join(RESULTS_PATH, model+'/'+method+'/'+drug+'_results.csv'))
      result = result_file[drug].to_numpy()

      result_class = result > 0.5 #can change this threshold
      confusion_matrix = metrics.confusion_matrix(truth, result_class)

      # (True Positive + True Negative) / Total Predictions
      Accuracy = metrics.accuracy_score(truth, result_class)
      # True Positive / (True Positive + False Positive)
      Precision = metrics.precision_score(truth, result_class)
      # True Positive / (True Positive + False Negative)
      Sensitivity_recall = metrics.recall_score(truth, result_class)
      # 2 * ((Precision * Sensitivity) / (Precision + Sensitivity)), Good for imbalanced datasets
      # F1_score = metrics.f1_score(truth, result_class)
      rocauc = metrics.roc_auc_score(truth,result) #Note: rocauc doesn't use the result_class

      evaluation_list.append([str(model)+'('+str(method)+')',drug,Accuracy,Precision,Sensitivity_recall,rocauc])

output = pd.DataFrame(evaluation_list)
output.to_csv(os.path.join(RESULTS_PATH, 'evaluation.csv'),header=False, index = False)
output


,0,1,2,3,4,5
0,Model,Drug,Accuracy,Precision,Recall,rocauc
1,SLE_results(PCA),AMK,0.973607,0.972644,1.0,0.948363
2,SLE_results(SMOTE),AMK,0.973607,0.972644,1.0,0.840625
3,SLE_results(column_modified),AMK,0.973607,0.972644,1.0,0.922917
4,SVM_results(PCA),AMK,0.973607,0.972644,1.0,0.876042
...,...,...,...,...,...,...
116,XGBoost_results(SMOTE),STR,0.888889,0.900533,0.962049,0.950906
117,XGBoost_results(column_modified),STR,0.896199,0.902827,0.969639,0.948144
118,logistic_regression_results(PCA),STR,0.935673,0.943119,0.975332,0.944379
119,logistic_regression_results(SMOTE),STR,0.928363,0.936131,0.973435,0.950519
